In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pre_processing as pp
import analysis as anl
import pca

category = 'test'

hNeg = True #if true, add negative bigrams for negative reviews
noun = False #if true, add nouns

X, Y, vocabulary = pp.bow(category, hNeg, noun)

print("Vocabulário possui " + str(len(vocabulary)) + " palavras!")

Vocabulário possui 1509 palavras!


In [2]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X2, Y2 = X[idx_perm, :], Y[idx_perm]

pTrain = 0.8

train_index, test_index = anl.stratified_holdOut(Y, pTrain)

Xtrain, Xval = X2[train_index, :], X2[test_index, :];
Ytrain, Yval = Y2[train_index], Y2[test_index];

In [3]:
Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
Xval = Xval[:, index]

#converte para outro tipo de matriz esparsa que facilita os cálculos do svm
Xtrain = Xtrain.tolil() 
Xval = Xval.tolil()

In [4]:
print("Número de features antes do chi-quadrado: " + str(len(vocabulary)))
print("----------------------------------------")
print("Número de features após chi-quadrado: " + str(len(new_vocabulary)))
print(new_vocabulary)
# with open('app/vocabulary.txt', 'w') as out_file:
#     for word in new_vocabulary:
#         if isinstance(word, tuple):
#             s = '('+word[0]+","+word[1]+')'+'\n'
#             out_file.write(s)
#         else:
#             out_file.write(word+'\n')

Número de features antes do chi-quadrado: 1509
----------------------------------------
Número de features após chi-quadrado: 5
['arab', 'entir', 'not', 'well', 'less']


In [5]:
import svmutil
from svmutil import svm_read_problem
from svmutil import svm_problem
from svmutil import svm_parameter
from svmutil import svm_train
from svmutil import svm_predict
from svmutil import svm_save_model

def gridSearch(X, Y, Xval, Yval):
    #inicializa as variáveis que deverão ser retornadas pela função
    custo = 1000
    gamma = 1000
    
    values = []
    for i in range(1, 10):
         for j in range(-1, 3):
            values.append(i/(10 ** j))
    
    acr = 0
    
    for i in values:
        for j in values:
            print(i)
            print(j)
            model = svm_train(Y, X, '-c %f -g %f -q' %(i, j))
            classes = svm_predict([], Xval, model, '-q')
            acuracia = np.sum(np.array(classes[0])==Yval)/len(Yval)
            if acr < acuracia:
                custo = i
                gamma = j
                acr = acuracia
                print(acr)
    
    return custo, gamma

In [6]:
def svm(Xtrain,Ytrain,Xval,Yval) :

    #bestC, bestGamma = gridSearch(Xtrain, Ytrain, Xval, Yval)

    bestC = 10
    bestGamma = 0.01
    
    kernel = 2 #kernel radial
    model = svm_train(Ytrain, Xtrain, '-c %f -t %d -g %f -q' %(bestC, kernel, bestGamma))
    
    #svm_save_model('app/test.model', model)
    
    classes = svm_predict([], Xval, model, '-q')
    acuracia = np.sum(np.array(classes[0])==Yval)/len(Yval)

    return np.array(classes[0], dtype="int"), acuracia

In [7]:
Ypred, acuracia = svm(Xtrain,Ytrain,Xval,Yval)
print("Acurácia é "+ str(acuracia*100))

Acurácia é 62.5


In [8]:
classes = np.unique(Y)
auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)


	Revocacao   Precisao   F-medida   Classe
	0.750       0.600      0.667      0
	0.500       0.667      0.571      1
	------------------------------------------------
	0.625       0.633      0.629      Média macro
	0.625       0.625      0.625      Média micro

	Acuracia: 0.625


In [9]:
def curva_aprendizado(X, Y, Xval, Yval):
   
    """
    Funcao usada gerar a curva de aprendizado.
  
    Parametros
    ----------
  
    X : matriz com os dados de treinamento
  
    Y : vetor com as classes dos dados de treinamento
  
    Xval : matriz com os dados de validação
  
    Yval : vetor com as classes dos dados de validação
  
    """

    # inicializa as listas que guardarao a performance no treinamento e na validacao
    perf_train = []
    perf_val = []

    classes = np.unique(Y)
    
    for i in range(10, len(Y), 1):
           
        Ypred, acuracia = svm(X[:i],Y[:i],X[:i],Y[:i])
        perf_train.append(acuracia)

        Ypred, acuracia = svm(X[:i],Y[:i],Xval,Yval)
        perf_val.append(acuracia)

    ##################################################################################
       
    # Define o tamanho da figura 
    plt.figure(figsize=(20,12))

    # Plota os dados
    plt.plot(perf_train, color='blue', linestyle='-', linewidth=1.5, label='Treino') 
    plt.plot(perf_val, color='red', linestyle='-', linewidth=1.5, label='Validação')

    # Define os nomes do eixo x e do eixo y
    plt.xlabel(r'# Qtd. de dados de treinamento',fontsize='x-large') 
    plt.ylabel(r'Acuracia',fontsize='x-large') 

    # Define o título do gráfico
    plt.title(r'Curva de aprendizado', fontsize='x-large')

    # Acrescenta um grid no gráfico
    plt.grid(axis='both')

    # Plota a legenda
    plt.legend()
    
    plt.show()



In [10]:
#curva_aprendizado(Xtrain, Ytrain, Xval, Yval)

In [23]:
#K-folds

import k_folds as kf

#Pega todos os tipos de classes 
classes = classes = np.unique(Y)

# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X3, Y3 = X[idx_perm, :], Y[idx_perm]

# separa os dados em k folds
nFolds = 5
folds = kf.stratified_kfolds(Y3, nFolds, classes)

k = 1
resultados=[] # cria uma lista vazia para guardar os resultados obtidos em cada fold
for train_index, test_index in folds:

    print('\n-----------\n%d-fold: \n-----------\n' % (k) )

    # se train_index ou test_index forem vazios, interrompe o laco de repeticao
    if len(train_index)==0 or len(test_index)==0: 
        print('\tErro: o vetor com os indices de treinamento ou o vetor com os indices de teste esta vazio')      
        break
        
    X_train, X_val = X3[train_index, :], X3[test_index, :];
    Y_train, Y_val= Y3[train_index], Y3[test_index];

    print(train_index)
    print(test_index)
    Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
    Xval = Xval[:, index]
    
    #converte para outro tipo de matriz esparsa que facilita os cálculos do svm
    Xtrain = Xtrain.tolil() 
    Xval = Xval.tolil()

    Ypred, acuracia = svm(Xtrain,Ytrain,Xval,Yval)
    
    auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)

    # adiciona os resultados do fold atual na lista de resultados
    resultados.append( auxResults )
    
    k = k + 1
    
print('\n-----------\n media-fold: \n-----------\n')
kf.mediaFolds( resultados, classes )


-----------
1-fold: 
-----------

[19, 20, 21, 23, 25, 29, 31, 32, 15, 18, 22, 24, 26, 27, 28, 30, 33, 34, 35, 38, 39, 40, 42, 43, 36, 37, 41, 45, 46, 48, 49, 50, 44, 47, 51, 52, 55, 56, 58, 59, 53, 54, 57, 60, 61, 65, 68, 69, 62, 63, 64, 66, 67, 73, 77, 70, 71, 72, 74, 75, 76, 78, 79]
[0, 1, 5, 10, 12, 13, 16, 17, 2, 3, 4, 6, 7, 8, 9, 11, 14]

	Revocacao   Precisao   F-medida   Classe
	0.750       0.600      0.667      0
	0.500       0.667      0.571      1
	------------------------------------------------
	0.625       0.633      0.629      Média macro
	0.625       0.625      0.625      Média micro

	Acuracia: 0.625

-----------
2-fold: 
-----------

[0, 1, 5, 10, 12, 13, 16, 17, 2, 3, 4, 6, 7, 8, 9, 11, 14, 33, 34, 35, 38, 39, 40, 42, 43, 36, 37, 41, 45, 46, 48, 49, 50, 44, 47, 51, 52, 55, 56, 58, 59, 53, 54, 57, 60, 61, 65, 68, 69, 62, 63, 64, 66, 67, 73, 77, 70, 71, 72, 74, 75, 76, 78, 79]
[19, 20, 21, 23, 25, 29, 31, 32, 15, 18, 22, 24, 26, 27, 28, 30]

	Revocacao   Precisao   F-